# 6. Finale Demo Applicatie

Dit notebook bevat de interactieve demo van ons schilderijen-classificatiemodel.
We gebruiken de library **Gradio** om een web-interface te maken waar gebruikers een eigen afbeelding kunnen uploaden.

Het model dat we gebruiken is **VGG16 met Transfer Learning**, getraind op de Vlaamse Supercomputer (VSC), met een nauwkeurigheid van ~94%.

## Imports en model bouwen

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
import gradio as gr
import numpy as np
from PIL import Image
import os

# 1. Instellingen
IMG_SIZE = (224, 224)
CLASS_NAMES = ["Mondriaan", "Picasso", "Rembrandt", "Rubens"] # Alfabetisch!
MODEL_PATH = "../models/transfer_model_vgg16.h5" # Pas aan naar jouw pad

# 2. Model Opbouwen
print("Model opbouwen...")

# Basis (VGG16)
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
conv_base.trainable = False

# Architectuur (Precies zoals in je script!)
inputs = tf.keras.Input(shape=(224, 224, 3))

# Augmentation (Precies deze 3, GEEN Contrast!)
x = layers.RandomFlip("horizontal")(inputs)
x = layers.RandomRotation(0.1)(x)
x = layers.RandomZoom(0.1)(x)

# Preprocessing (Zit in het model!)
x = tf.keras.applications.vgg16.preprocess_input(x)

# De rest
x = conv_base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(4, activation='softmax')(x)

model = models.Model(inputs, outputs)

# 3. Gewichten laden
if os.path.exists(MODEL_PATH):
    print(f"Gewichten laden van {MODEL_PATH}...")
    try:
        # We gebruiken by_name=True om versieconflicten te omzeilen
        # skip_mismatch zorgt dat hij niet crasht op kleine naamverschillen in de inputlaag
        model.load_weights(MODEL_PATH, by_name=True, skip_mismatch=True)
        print("✅ Gewichten succesvol geladen!")
        
        # Check: Print even een gewicht om te zien of het geen 0 is
        # (Als dit 0 is, is het laden mislukt)
        check_w = model.layers[-1].get_weights()[0][0]
        print(f"Check gewicht sample: {check_w}")
        
    except Exception as e:
        print(f"❌ Fout bij laden gewichten: {e}")
else:
    print(f"⚠️ Bestand '{MODEL_PATH}' niet gevonden. Zet het .h5 bestand in de juiste map.")

Model opbouwen...
Gewichten laden van ../models/transfer_model_vgg16.h5...
✅ Gewichten succesvol geladen!
Check gewicht sample: [-0.14868304  0.11420235  0.01047193 -0.0933736 ]


/home/mathieu/Documenten/Deep_Learning_Schilderijen/.venv/lib/python3.13/site-packages/keras/src/legacy/saving/legacy_h5_format.py:525: UserWarning: Skipping loading weights for layer #6 (named vgg16)due to mismatch in shape for weight block1_conv1/kernel. Weight expects shape (3, 3, 3, 64). Received saved weight with shape (3, 3, 512, 512)
  _set_weights(
/home/mathieu/Documenten/Deep_Learning_Schilderijen/.venv/lib/python3.13/site-packages/keras/src/legacy/saving/legacy_h5_format.py:525: UserWarning: Skipping loading weights for layer #6 (named vgg16)due to mismatch in shape for weight block1_conv1/bias. Weight expects shape (64,). Received saved weight with shape (512,)
  _set_weights(
/home/mathieu/Documenten/Deep_Learning_Schilderijen/.venv/lib/python3.13/site-packages/keras/src/legacy/saving/legacy_h5_format.py:525: UserWarning: Skipping loading weights for layer #6 (named vgg16)due to mismatch in shape for weight block1_conv2/kernel. Weight expects shape (3, 3, 64, 64). Received

## De voorspelfunctie

Hier maken we de functie die Gradio gaat gebruiken

In [5]:
def classify_image(image):
    if image is None:
        return None
    
    # 1. Afbeelding voorbereiden
    # Gradio geeft een numpy array, wij willen een PIL image om te resizen
    img = Image.fromarray(image.astype('uint8'), 'RGB')
    img = img.resize(IMG_SIZE)
    
    # Omzetten naar array en batch dimensie toevoegen (1, 224, 224, 3)
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    
    # 2. Voorspellen
    prediction = model.predict(img_array, verbose=0)
    
    # 3. Resultaat omzetten naar dictionary {Label: Kans}
    # prediction[0] bevat de 4 kansen (bijv. [0.01, 0.90, 0.05, 0.04])
    result_dict = {CLASS_NAMES[i]: float(prediction[0][i]) for i in range(len(CLASS_NAMES))}
    
    return result_dict

## De interface starten

In [6]:
# Interface bouwen
interface = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(label="Upload een schilderij"),
    outputs=gr.Label(num_top_classes=4, label="Voorspelling"),
    title="🎨 Schilderijen Herkenner AI",
    description="Upload een foto van een schilderij en het model vertelt of het een Mondriaan, Picasso, Rembrandt of Rubens is.",
    theme="default"
)

# Start de app
interface.launch(share=True)

/home/mathieu/Documenten/Deep_Learning_Schilderijen/.venv/lib/python3.13/site-packages/gradio/interface.py:171: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  super().__init__(


* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://afaf5a101afd6684ca.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Created dataset file at: .gradio/flagged/dataset1.csv
